In [ ]:
from random import shuffle
import os
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import svm
import cv2
import imutils
import pandas as pd
import pickle
import numpy as np

def preprocess(img):
    img = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=5)
    return img
    

letters = [chr(97+i) for i in range(26)]
folder_path = "handgesturedataset/"
models = {}
accs =[]
zooms = [24,32,40,48,56]
num_angles = 30
for letter in letters:
    X,Y = [],[]
    seenLetters = set()
    trues = 0
    falses = 0
    files =os.listdir(folder_path)
    shuffle(files)
    for file in files:
        if trues == 40:
            break
        if "_"+letter+"_" not in file and falses > 35:
            continue
        if file == ".DS_Store":
            continue
        unique = file.split('seg')[0]
        if unique in seenLetters:
            continue
        elif "_"+letter+"_" not in unique:
            seenLetters.add(unique)
            
        orig = cv2.imread(folder_path+file)
        count = 0
        for flip in [orig, cv2.flip(orig,0)]:
            for angle in range(0,360,num_angles):
                img = imutils.rotate(flip,angle)
                img = cv2.resize(img, (64,64))
                img = preprocess(img)
                for zoom in zooms:
                    left =(64-zoom)//2
                    right = zoom + left
                    zoomed = img[left:right,left:right]
                    X.append(cv2.resize(zoomed,(64,64)).flatten())
                    count +=1
                X.append(img.flatten())
                count +=1
            X.append(img.flatten())
            count +=1
        if "_"+letter+"_" in file:
            Y.extend([1]*count)
            trues += 1
            
        else:
            Y.extend([0]*count)
            falses += 1
        
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)
#     model = svm.SVC(kernel='linear', C=1)
    model = LogisticRegression()
    print("Training letter",letter,"...")
    print("number of trainig:", len(X_train))
    model.fit(X_train,Y_train)
    accs.append(model.score(X_test,Y_test))
    models[letter.upper()] = model
    print(letter, accs[-1])
    pickle.dump(model, open("models/"+letter+".model", 'wb'))



Training letter a ...
number of trainig: 7708


/Users/shreysamdani/anaconda3/envs/ieor135/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


a 0.8153526970954357
Training letter b ...
number of trainig: 7708


/Users/shreysamdani/anaconda3/envs/ieor135/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


b 0.7717842323651453
Training letter c ...
number of trainig: 7708


...

/Users/shreysamdani/anaconda3/envs/ieor135/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


d 0.7489626556016598
Training letter e ...
number of trainig: 7708


/Users/shreysamdani/anaconda3/envs/ieor135/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


e 0.7878630705394191
Training letter f ...
number of trainig: 7708


/Users/shreysamdani/anaconda3/envs/ieor135/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [44]:
import pickle
for letter, mod in models.items():
    pickle.dump(mod, open("models/"+letter+".model", 'wb'))